# Creates a Redshift Cluster using the AWS python SDK 

In [1]:
import pandas as pd
import boto3
import json
import time

# STEP 0: Make sure you have an AWS secret and access key

- Create a new IAM user in your AWS account
- Give it `AdministratorAccess`, From `Attach existing policies directly` Tab
- Take note of the access key and secret 
- Edit the file `dwh.cfg` in the same folder as this notebook and fill
<font color='red'>
<BR>
\[AWS\]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>
<font/>


# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
NUM_NODES          = config.get("CLUSTER","NUM_NODES")
NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

CLUSTER_IDENTIFIER     = config.get("CLUSTER","CLUSTER_IDENTIFIER")
DB_NAME                = config.get("CLUSTER_ID","DB_NAME")
DB_USER                = config.get("CLUSTER_ID","DB_USER")
DB_PASSWORD            = config.get("CLUSTER_ID","DB_PASSWORD")
DB_PORT                = config.get("CLUSTER_ID","DB_PORT")

ARN                    = config.get("IAM_ROLE", "ARN")

S3_LOG_DATA            = config.get("S3", "LOG_DATA")
S3_LOG_JSONPATH        = config.get("S3", "LOG_JSONPATH")
S3_SONG_DATA           = config.get("S3", "SONG_DATA")

(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT", "ARN"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, ARN]
             })

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,dwhCluster
4,DB_NAME,dwh
5,DB_USER,dwhuser
6,DB_PASSWORD,Passw0rd
7,DB_PORT,5439
8,ARN,arn:aws:iam::796040616481:role/dwhRole-Sabathh


## Create clients for EC2, S3, IAM, and Redshift

In [3]:
import boto3

ec2 = boto3.resource('ec2',
                      region_name='us-west-2',
                      aws_access_key_id=KEY,
                      aws_secret_access_key=SECRET)

s3 = boto3.resource('s3', 
                     region_name='us-west-2',
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

iam = boto3.client('iam', 
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

redshift = boto3.client('redshift', 
                         region_name='us-west-2', 
                         aws_access_key_id=KEY, 
                         aws_secret_access_key=SECRET)

## Test access to S3 bucket 'udacity-dend' 

In [4]:
udacity_dend_bucket =  s3.Bucket("udacity-dend")

# Iterate over bucket objects located in 'song_data/A/A/A'
for obj in udacity_dend_bucket.objects.filter(Prefix='song_data/A/A/A'):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAFD128F92F423A.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAGR128F425B14B.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAHD128F426

In [5]:
# Iterate over bucket objects located in 'log_data/A/A/A'
for obj in udacity_dend_bucket.objects.filter(Prefix='log_data/2018/11/2018-11-0'):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-09-events.json')


## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [6]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DB_USER,
        Description='Allows Redshift clusters to call AWS services on your behalf.',
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )
    

except Exception as e:
    print(e)

1.1 Creating a new IAM Role
An error occurred (ValidationError) when calling the CreateRole operation: The specified value for roleName is invalid. It must contain only alphanumeric characters and/or the following: +=,.@_-


In [11]:
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DB_USER,
                  PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                  )['ResponseMetadata']['HTTPStatusCode']


1.2 Attaching Policy


NoSuchEntityException: An error occurred (NoSuchEntity) when calling the AttachRolePolicy operation: The role with name dwhRole-Sabathh cannot be found.

In [ ]:
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=ARN)['Role']['Arn']

print(roleArn)

## STEP 2:  Redshift Cluster

- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [ ]:
try:
    response = redshift.create_cluster(        
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- This block will keep on running until the cluster status becomes `Available`

In [ ]:
from IPython.display import clear_output, display

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

while True:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
    display(prettyRedshiftProps(myClusterProps))
    if myClusterProps['ClusterStatus'] == 'available':
        break
    time.sleep(5)
    clear_output(wait=True)

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',  
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DWH_ENDPOINT, DB_PORT,DB_NAME)
print(conn_string)
%sql $conn_string

## STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE you want to delete the redshift cluster <br/> 

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
time.sleep(5)
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

- run this block several times to check if the cluster is really deleted

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=ARN, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=ARN)
#### CAREFUL!!